In [1]:
import pandas as pd
import json
import networkx as nx
from networkx.algorithms import bipartite
from tqdm import tqdm
import statsmodels.api as sm
import statsmodels.formula.api as smf
pd.options.mode.chained_assignment = None 

In [6]:
movie_metadata_header = [
    'ID',
    'Freebase movie ID',
    'Movie name',
    'Movie release date',
    'Movie box office revenue',
    'Movie runtime',
    'Movie languages',
    'Movie countries',
    'Movie genres'
]

character_metadata_header = [
    'ID',
    'Freebase movie ID',
    'Movie release date',
    'Character Name',
    'Actor DOB',
    'Actor gender',
    'Actor height',
    'Actor ethnicity',
    'Actor Name',
    'Actor age at movie release',
    'Freebase character map',
    'Freebase character ID',
    'Freebase actor ID'
]

In [25]:
# read oscar dataset
oscar_dataset = pd.read_csv('the_oscar_award.csv')
oscar_dataset = oscar_dataset.rename(columns = {'name': 'Actor Name','film' : 'Movie name','year_film':'Movie release year'})
# only take actor/actress Awards from all categories
oscar_nominees = oscar_dataset[oscar_dataset['category'].str.contains('ACTOR') | oscar_dataset['category'].str.contains('ACTRESS')].reset_index(drop=True)
oscar_winners = oscar_nominees[oscar_nominees['winner'] == True]
oscar_nominees.to_csv('oscar_nominees.csv')
# we look for the first win and first nomination of every actor
actor_first_win = oscar_winners.groupby('Actor Name').agg({'Movie release year': 'min',
                                                           'year_ceremony'     : 'min'}).reset_index()
actor_first_nomination =  oscar_nominees.groupby('Actor Name').agg({'Movie release year': 'min',
                                                                    'year_ceremony'     : 'min'}).reset_index()
actor_first_win = actor_first_win.rename(columns = {'year_ceremony' :'first win date'}).drop(columns=['Movie release year'])
actor_first_nomination = actor_first_nomination.rename(columns = {'year_ceremony' :'first nomination date'}).drop(columns=['Movie release year'])
best_actors = pd.merge(actor_first_nomination,actor_first_win,on = 'Actor Name', how = 'left')
best_actors.fillna(9999)

,Actor Name,first nomination date,first win date
0,Abigail Breslin,2007,9999.0
1,Adam Driver,2019,9999.0
2,Adolph Caesar,1985,9999.0
3,Adolphe Menjou,1931,9999.0
4,Adriana Barraza,2007,9999.0
...,...,...,...
930,Winona Ryder,1994,9999.0
931,Woody Allen,1978,9999.0
932,Woody Harrelson,1997,9999.0
933,Yalitza Aparicio,2019,9999.0


In [8]:
character_metadata = pd.read_csv(
    'MovieSummaries/character.metadata.tsv',
    delimiter='\t',
    names=character_metadata_header
)
# in order to compare release years
character_metadata['Movie release year'] = pd.to_numeric(character_metadata['Movie release date'].str.split('-').str[0],errors = 'coerce')
actors_in_movies = character_metadata[['ID','Character Name','Actor Name','Movie release year']]

In [9]:
oscars_in_movies = pd.merge(actors_in_movies,best_actors,on = 'Actor Name',how = 'left')
oscars_in_movies.fillna({'first nomination date': 9999,'first win date': 9999,'ceremony':0, 'winner':False},inplace= True)
oscars_in_movies

,ID,Character Name,Actor Name,Movie release year,first nomination date,first win date
0,975900,Akooshay,Wanda De Jesus,2001.0,9999.0,9999.0
1,975900,Lieutenant Melanie Ballard,Natasha Henstridge,2001.0,9999.0,9999.0
2,975900,Desolation Williams,Ice Cube,2001.0,9999.0,9999.0
3,975900,Sgt Jericho Butler,Jason Statham,2001.0,9999.0,9999.0
4,975900,Bashira Kincaid,Clea DuVall,2001.0,9999.0,9999.0
...,...,...,...,...,...,...
450664,913762,Elensh,Dorothy Elias-Fahn,1992.0,9999.0,9999.0
450665,913762,Hibiki,Jonathan Fahn,1992.0,9999.0,9999.0
450666,28308153,NaN,David Hemmings,1957.0,9999.0,9999.0
450667,28308153,NaN,Roberta Paterson,1957.0,9999.0,9999.0


In [10]:
oscars_in_movies['nominated'] = oscars_in_movies['Movie release year'] > oscars_in_movies['first nomination date'].astype(int)
oscars_in_movies['winner'] = oscars_in_movies['Movie release year'] > oscars_in_movies['first win date'].astype(int)
# oscars_in_movies.groupby('ID')['ID'].count()
oscars_per_movie = oscars_in_movies.groupby('ID').agg({'winner':'sum',
                                                       'nominated':'sum'}).reset_index()
oscars_per_movie.sort_values(by = 'winner',ascending= False)

,ID,winner,nominated
57346,31232694,11,12
24873,10821674,10,22
37485,19187200,7,13
1563,327427,6,15
631,101398,5,8
...,...,...,...
23223,9894771,0,0
23224,9894921,0,0
23225,9894962,0,0
23226,9895416,0,0


In [11]:
movie_metadata = pd.read_csv(
    'MovieSummaries/movie.metadata.tsv',
    delimiter='\t',
    names=movie_metadata_header
)
# in order to compare release years
movie_metadata['Movie release year'] = pd.to_numeric(movie_metadata['Movie release date'].str.split('-').str[0],errors = 'coerce')


## IMDB

In [12]:
imdb_title_akas_fname = 'additional_data/imdb_data/title_akas.tsv'
imdb_title_ratings_fname = 'additional_data/imdb_data/title_ratings.tsv'
imdb_title_ratings = pd.read_csv(imdb_title_ratings_fname,sep = "\t")
imdb_title_akas = pd.read_csv(imdb_title_akas_fname,sep = "\t")
imdb_title_akas =imdb_title_akas[imdb_title_akas['isOriginalTitle']==1]
imdb_title_akas = imdb_title_akas[['titleId','title']]
ratings= pd.merge(imdb_title_akas,imdb_title_ratings,left_on = 'titleId',right_on= 'tconst',how = 'inner')
ratings = ratings[['title','averageRating','numVotes']]


C:\Users\benmu\AppData\Local\Temp\ipykernel_18408\2172311594.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_title_akas = pd.read_csv(imdb_title_akas_fname,sep = "\t")


In [13]:
# Final merge for movie metadata
movie_metadata_with_rating = pd.merge(movie_metadata,ratings,left_on='Movie name',right_on='title',how ='inner')
movie_metadata_with_rating = movie_metadata_with_rating.drop(columns=['title'])
movie_metadata_with_rating = movie_metadata_with_rating.drop_duplicates(subset=['Movie name'],keep = False)
movie_metadata_with_rating

,ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,Movie release year,averageRating,numVotes
0,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",1988.0,5.7,40
3,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/04t36"": ""Musical"", ""/m/01z4y"": ""Comedy"", ...",1938.0,6.9,2160
10,11250635,/m/02r52hc,The Mechanical Monsters,NaN,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06n90"": ""Science Fiction"", ""/m/03k9fj"": ""...",NaN,7.4,1441
11,77856,/m/0kcn7,Mary Poppins,1964-08-27,102272727.0,139.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3myq"": ""Children's/Family"", ""/m/04t36""...",1964.0,7.8,173375
12,32456683,/m/0gyryjt,Die Fahne von Kriwoj Rog,1967,NaN,108.0,"{""/m/04306rv"": ""German Language""}","{""/m/03f2w"": ""German Democratic Republic""}",{},1967.0,7.7,22
...,...,...,...,...,...,...,...,...,...,...,...,...
35208,1673588,/m/05mc7l,The Brother from Another Planet,1984-09-07,NaN,104.0,"{""/m/02h40lc"": ""English Language"", ""/m/06nm1"":...","{""/m/09c7w0"": ""United States of America""}","{""/m/06n90"": ""Science Fiction"", ""/m/0219x_"": ""...",1984.0,6.8,6181
35209,19525452,/m/04n12_2,Pigen og vandpytten,1958-11-24,NaN,97.0,"{""/m/0295r"": ""Danish Language""}","{""/m/0k6nt"": ""Denmark""}","{""/m/0hqxf"": ""Family Film"", ""/m/01g6gs"": ""Blac...",1958.0,5.9,89
35210,20123336,/m/04y6qzy,The Race for Space,1959,NaN,55.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0jtdp"": ""Documentary""}",1959.0,6.5,45
35211,7761830,/m/0kvgqb,Spaced Invaders,1990,15369573.0,100.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3mt0"": ""Alien Film"", ""/m/06n90"": ""Scie...",1990.0,5.3,3743


In [14]:
movies = pd.merge(movie_metadata_with_rating,oscars_per_movie,on = 'ID', how = 'inner')
movies['Movie genres']= movies['Movie genres'].apply(lambda row : list(json.loads(row).values()))
movies['Movie languages']= movies['Movie languages'].apply(lambda row : list(json.loads(row).values()))
movies['Movie countries']= movies['Movie countries'].apply(lambda row : list(json.loads(row).values()))
movies['Movie release era'] = movie_metadata['Movie release year']//10
# movies['Movie genres']

In [30]:
## remove data without country or language
final_movies = movies[(movies['Movie countries'].str.len() != 0) & (movies['Movie languages'].str.len() != 0) ]
## take first country and first language from every movie
final_movies['Movie languages']= final_movies['Movie languages'].apply(lambda row : row[0])
final_movies['Movie countries']= final_movies['Movie countries'].apply(lambda row : row[0])
final_movies = final_movies.drop(columns = ['Freebase movie ID','Movie box office revenue','Movie release date', 'Movie runtime'])
final_movies.to_csv('movies_with_rating.csv',index=False)
final_movies

,ID,Movie name,Movie languages,Movie countries,Movie genres,Movie release year,averageRating,numVotes,winner,nominated,Movie release era
0,28463795,Brun bitter,Norwegian Language,Norway,"[Crime Fiction, Drama]",1988.0,5.7,40,0,0,200.0
1,10408933,Alexander's Ragtime Band,English Language,United States of America,"[Musical, Comedy, Black-and-white]",1938.0,6.9,2160,0,0,200.0
2,11250635,The Mechanical Monsters,English Language,United States of America,"[Science Fiction, Adventure, Animation, Short ...",NaN,7.4,1441,0,0,198.0
3,77856,Mary Poppins,English Language,United States of America,"[Children's/Family, Musical, Fantasy, Comedy, ...",1964.0,7.8,173375,0,4,198.0
4,32456683,Die Fahne von Kriwoj Rog,German Language,German Democratic Republic,[],1967.0,7.7,22,0,0,198.0
...,...,...,...,...,...,...,...,...,...,...,...
11723,26219108,The Leopard Woman,Silent film,United States of America,"[Silent film, Romance Film]",NaN,5.7,48,0,0,194.0
11724,54540,Coming to America,English Language,United States of America,"[Romantic comedy, Comedy of manners, Drama, Co...",1988.0,7.1,208755,1,3,192.0
11725,1673588,The Brother from Another Planet,English Language,United States of America,"[Science Fiction, Indie, Cult, Drama, Comedy, ...",1984.0,6.8,6181,0,0,192.0
11726,19525452,Pigen og vandpytten,Danish Language,Denmark,"[Family Film, Black-and-white]",1958.0,5.9,89,0,0,197.0


In [16]:
movies_with_oscar_winner = final_movies[final_movies['winner'] > 0]
movies_with_oscar_nomination = final_movies[(final_movies['nominated'] > 0) & (final_movies['winner'] == 0) ]
movies_wo_oscar = final_movies[final_movies['nominated'] == 0]
print('number of movies with at least one oscar winner: '+ str(movies_with_oscar_winner.shape[0]))
print('number of movies with at least one oscar nomination: '+ str(movies_with_oscar_nomination.shape[0]))
print('number of movies with no oscar nomination: '+ str(movies_wo_oscar.shape[0]))


number of movies with at least one oscar winner: 1177
number of movies with at least one oscar nomination: 1449
number of movies with no oscar nomination: 7509


## Movies with Oscar winners
## VS
## Movies without any Oscar Nomination

In [17]:
comparator_winner_vs_no_nomination = pd.merge(
    movies_with_oscar_winner,
    movies_wo_oscar,
    on=['Movie release era','Movie languages','Movie countries'],
    suffixes=("_1", "_2")
)
comparator_winner_vs_no_nomination

,ID_1,Movie name_1,Movie languages,Movie countries,Movie genres_1,Movie release year_1,averageRating_1,numVotes_1,winner_1,nominated_1,Movie release era,ID_2,Movie name_2,Movie genres_2,Movie release year_2,averageRating_2,numVotes_2,winner_2,nominated_2
0,1364293,They Knew What They Wanted,English Language,United States of America,"[Romance Film, Marriage Drama, Drama, Black-an...",1940.0,6.1,443,1,2,193.0,1305819,C.H.U.D.,"[Monster movie, Natural horror films, Science ...",1984.0,5.6,12907,0,0
1,1364293,They Knew What They Wanted,English Language,United States of America,"[Romance Film, Marriage Drama, Drama, Black-an...",1940.0,6.1,443,1,2,193.0,4453753,Begotten,"[Cult, Horror, Fantasy, Surrealism, Supernatural]",1991.0,5.6,11004,0,0
2,1364293,They Knew What They Wanted,English Language,United States of America,"[Romance Film, Marriage Drama, Drama, Black-an...",1940.0,6.1,443,1,2,193.0,7121282,The Wild Chase,"[Short Film, Family Film, Animation]",1965.0,6.2,424,0,0
3,1364293,They Knew What They Wanted,English Language,United States of America,"[Romance Film, Marriage Drama, Drama, Black-an...",1940.0,6.1,443,1,2,193.0,418154,Basic Training,"[Parody, Sex comedy, Comedy, Indie]",1985.0,4.1,474,0,0
4,1364293,They Knew What They Wanted,English Language,United States of America,"[Romance Film, Marriage Drama, Drama, Black-an...",1940.0,6.1,443,1,2,193.0,23283544,Granite Hotel,"[Short Film, Comedy film, Animation]",1940.0,5.6,49,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411282,369508,Gandhi,Hindi Language,India,"[British Empire Film, History, Historical fict...",1982.0,8.1,232113,1,4,201.0,10776379,Apradh,"[Musical, Drama, Bollywood, World cinema]",1972.0,7.1,83,0,0
411283,369508,Gandhi,Hindi Language,India,"[British Empire Film, History, Historical fict...",1982.0,8.1,232113,1,4,201.0,20903398,Aaj Ki Taaza Khabar,"[Action, Bollywood]",1973.0,7.7,103,0,0
411284,369508,Gandhi,Hindi Language,India,"[British Empire Film, History, Historical fict...",1982.0,8.1,232113,1,4,201.0,8658217,Sujata,"[Romance Film, Drama, World cinema, Bollywood]",1959.0,7.6,388,0,0
411285,369508,Gandhi,Hindi Language,India,"[British Empire Film, History, Historical fict...",1982.0,8.1,232113,1,4,201.0,2298412,Gentleman,"[Crime Fiction, Musical, Drama, Tamil cinema, ...",1993.0,6.8,479,0,0


In [18]:
## see how many genres are in common between two movies
intersection = lambda list1, list2 :len(list(set(list1).intersection(list2)))
comparator_winner_vs_no_nomination['genres_matches'] = comparator_winner_vs_no_nomination[['Movie genres_1','Movie genres_2']].apply(lambda x: intersection(*x) ,axis = 1)
# only keep movie pairs with at least one genre in common
comparator_with_matches = comparator_winner_vs_no_nomination[comparator_winner_vs_no_nomination['genres_matches']>0]


In [19]:
## we use perfect matching here in order to have a maximum 1-on-1 matching between movies with oscar winners 
## and movies without any oscar nomination

# Initialise graph
graph = nx.Graph()
graph.add_nodes_from(comparator_with_matches['ID_1'], bipartite=0)
graph.add_nodes_from(comparator_with_matches['ID_2'], bipartite=1)
# add edges between movies with at least one oscar nominee and movies with no oscar nomination
# with weight equal the number of genres in common between the two movies
graph.add_weighted_edges_from(
    [(row['ID_1'], row['ID_2'], row['genres_matches']) for index,row in tqdm(comparator_with_matches.iterrows(),total =comparator_with_matches.shape[0])], weight='weight')
# maximize the weight in order to have the best matching between movies 
# matching them with most genres in common while trying to have maximum cardinality
perfect_matching = nx.max_weight_matching(graph)

100%|██████████| 170389/170389 [00:06<00:00, 25493.76it/s]


In [20]:
# we merge all movies pairs and the perfect_matching
matched_movies = pd.DataFrame(perfect_matching)
matched_movies = matched_movies.rename(columns = {0:"ID_2", 1: "ID_1"})
matched_movies = pd.merge(matched_movies,comparator_winner_vs_no_nomination, on = ['ID_1','ID_2'],how ='inner')
print('The number of matched pairs is: {}'.format(len(matched_movies)))
# print('The number of dropped athletes is : {}'.format(len(ja  )-2*len(matched_athletes)))
# print('The number of dropped athletes before is : {}'.format(len(athlete_before)-len(matched_athletes)))
# print('The number of dropped athletes after is : {}'.format(len(athlete_after)-len(matched_athletes)))anese_athletes_cutoff

The number of matched pairs is: 958


In [21]:
(matched_movies['averageRating_1']>matched_movies['averageRating_2']).sum()/matched_movies.shape[0]*100


58.55949895615866

In [22]:
## split id1 and id2
matched_movies_1 = matched_movies[matched_movies.filter(regex='_1|Movie release era|Movie languages|Movie countries').columns]
matched_movies_2 = matched_movies[matched_movies.filter(regex='_2|Movie release era|Movie languages|Movie countries').columns]
# removing prefix for column names so we can have the same column names for matched_movies_1 and matched_movies_2
matched_movies_1.columns = matched_movies_1.columns.str.rstrip('_1')
matched_movies_2.columns = matched_movies_2.columns.str.rstrip('_2')
# concatenating the two dataframes into one
final_matched_movies = pd.concat([matched_movies_1,matched_movies_2])
# adding is_after value
final_matched_movies['OscarWinner'] =(final_matched_movies['winner']>0).astype(int)
final_matched_movies

,ID,Movie name,Movie languages,Movie countries,Movie genres,Movie release year,averageRating,numVotes,winner,nominated,Movie release era,OscarWinner
0,3248340,Kung Fu Panda,English Language,United States of America,"[Adventure, Children's/Family, Computer Animat...",2008.0,7.2,422,2,3,197.0,1
1,385742,The Fall of the Roman Empire,English Language,United States of America,"[Sword and Sandal, Historical fiction, Epic, P...",1964.0,6.7,9355,2,5,201.0,1
2,14335277,It's Love I'm After,English Language,United States of America,"[Romantic comedy, Romance Film, Screwball come...",1937.0,7.4,2510,1,2,200.0,1
3,10916960,The Young Doctors,English Language,United States of America,"[Melodrama, Drama, Medical fiction]",1961.0,6.8,563,1,1,199.0,1
4,363929,Star 80,English Language,United States of America,"[Crime Fiction, Biographical film, Biography, ...",1983.0,6.8,7073,1,3,200.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
953,24035981,Doll Face,English Language,United States of America,"[Drama, Romance Film, Black-and-white, Comedy,...",1946.0,2.0,157,0,0,199.0,0
954,5876575,K2,English Language,United Kingdom,"[Action/Adventure, Sports, Drama, Adventure, F...",1992.0,6.2,6894,0,0,197.0,0
955,76313,Safety Last!,English Language,United States of America,"[Thriller, Silent film, Indie, Black-and-white...",1923.0,8.1,20955,0,0,200.0,0
956,9358839,Heidi's Song,English Language,United States of America,"[Musical, Children's, Drama, Animation]",1982.0,6.2,309,0,0,201.0,0


In [23]:
# linear regression formula
formula = "averageRating ~ OscarWinner"
df = final_matched_movies[['averageRating','OscarWinner']]
#train the linear regression with our final matched movies dataframe
mod = smf.ols(formula=formula, data= df).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:          averageRating   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     40.29
Date:                Sun, 18 Dec 2022   Prob (F-statistic):           2.73e-10
Time:                        12:41:05   Log-Likelihood:                -2851.2
No. Observations:                1916   AIC:                             5706.
Df Residuals:                    1914   BIC:                             5718.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       6.1552      0.035    177.686      